# Centroids Reconstruction on MNIST / Fashion MNIST datasets

In [1]:
import tensorflow as tf
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.pyplot as plt

If MNIST dataset is used, run:

In [2]:
mnist = tf.keras.datasets.mnist.load_data(path='mnist.npz')

Else if Fashion MNIST is used, run:

In [25]:
mnist = tf.keras.datasets.fashion_mnist.load_data()

Divide the data into test set and training set:

In [26]:
data_total = np.reshape(mnist[0][0], [-1,28*28])
maxx = np.reshape(np.amax(data_total, axis = -1), [-1,1])
print(maxx.shape)
data_total = np.reshape(data_total/maxx, [-1,28,28,1])
labels_total = mnist[0][1]
data_train, data_val, labels_train, labels_val = train_test_split(data_total, labels_total, test_size = 0.25, random_state = 42)
data_test = np.reshape(mnist[1][0], [-1,28*28])
maxx = np.reshape(np.amax(data_test, axis = -1), [-1,1])
data_test = np.reshape(data_test/maxx, [-1,28,28,1])
data_test.shape

(60000, 1)


(10000, 28, 28, 1)

Here we use test set:

In [27]:
data_val=data_test
labels_val=mnist[1][1]

Insert the name of the trained model:

In [28]:
ex_name ="hyperopt_100_8-8_2019-09-29_18851"

Create model path:

In [29]:
modelpath = "../models/{}/{}.ckpt".format(ex_name, ex_name)
som_dim=[8,8]

Run the trained model on test set:

In [ ]:
tf.reset_default_graph()
som_dim=[8,8]
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    saver = tf.train.import_meta_graph(modelpath+".meta")
    saver.restore(sess, modelpath)
    graph = tf.get_default_graph()
    is_t = graph.get_tensor_by_name("is_training/is_training:0")
    z_p = graph.get_tensor_by_name("reconstruction_e/decoder/z_e:0")
    x_hat = graph.get_tensor_by_name("reconstruction_e/decoder/x_hat:0")
    k = graph.get_tensor_by_name("k/k:0")
    z_q = graph.get_tensor_by_name("z_q/z_q:0")
    z_e = graph.get_tensor_by_name("sample_z_e/z_e:0")
    x = graph.get_tensor_by_name("inputs/x:0")
    q = graph.get_tensor_by_name("q/distribution/q:0")
    embeddings = graph.get_tensor_by_name("embeddings/embeddings:0")

    k_all = []
    test_rec_all = []
    test_mse_all = []
    labels_val_all = []
    z_q_all = []
    z_e_all = []
    print("Evaluation...")
    
    z_q_all = sess.run(z_q, feed_dict={x: data_val})
    q = sess.run(q, feed_dict={x:data_val})
    z_e_all = sess.run(z_e, feed_dict={x: data_val})
    k_all = sess.run(k, feed_dict={x: data_val})
    embeddings_e = sess.run(embeddings, feed_dict={x: data_val})
    x_hat_all = np.zeros((som_dim[0],som_dim[1],28,28))
    for i in range(som_dim[0]):
        for j in range(som_dim[1]):
            x_hat_o = sess.run(tf.sigmoid(x_hat), feed_dict={is_t: False, x: np.zeros((1, 28,28,1)), z_p: np.expand_dims(embeddings_e[i,j], 0)})
            x_hat_all[i,j] = np.reshape(x_hat_o, (28,28))

INFO:tensorflow:Restoring parameters from ../models/hyperopt_100_8-8_2019-09-29_18851/hyperopt_100_8-8_2019-09-29_18851.ckpt
Evaluation...


Reconstruct the input:

In [ ]:
fig, axes = plt.subplots(8, 8, figsize=(20,20))
for i in range(8):
    for j in range(8):
        axes[i,j].imshow(x_hat_all[i,j], cmap='gray')